In [2]:
import sys, os, platform
import h5py, tarfile
import numpy as np
from pandas import Series, DataFrame

In [3]:
h5filepath = 'e:/AShareSample/ETF/RFrames_SH510050.h5'
# h5filepath = '../out/RFrames_SZ159948.h5'
h5file = h5py.File(h5filepath, 'r')
h5file.keys()

<KeysViewHDF5 ['ReplayFrame:DQN_Cnn1Dx4.1548_3.F0001', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0002', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0003', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0004', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0005', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0006', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0007', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0008', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0009', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0010', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0011', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0012', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0013', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0014', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0015', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0016', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0017', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0018', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0019', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0020', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0021', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0022', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0023', 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0024', 'ReplayFrame:DQN_Cnn1Dx4.

In [4]:
framesInHd5 = []
for name in h5file.keys():
    if 'ReplayFrame:' == name[:len('ReplayFrame:')] :
        framesInHd5.append(name)

if len(framesInHd5)>3:
    del framesInHd5[0]
    del framesInHd5[-1]

framesInHd5

['ReplayFrame:DQN_Cnn1Dx4.1548_3.F0002',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0003',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0004',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0005',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0006',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0007',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0008',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0009',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0010',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0011',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0012',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0013',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0014',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0015',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0016',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0017',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0018',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0019',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0020',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0021',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0022',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0023',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0024',
 'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0025',
 'ReplayFrame:DQ

In [8]:
fname = framesInHd5[20]
fname

'ReplayFrame:DQN_Cnn1Dx4.1548_3.F0022'

In [31]:
H5_COLS = 'state,action,reward,next_state,done'
samplePool = {
            'state':[],
            'action':[],
            'reward':[],
            'next_state':[],
            'done':[],
        }

frame = h5file[fname]
statebatch = np.array(frame['state'].value)
actionbatch = np.array(frame['action'].value)
rewardbatch = np.array(frame['reward'].value)
#for col in H5_COLS.split(',') :
#    incrematal = list(frame[col].value)
#    samplePerFrame = len(incrematal)
#    samplePool[col] += incrematal
statebatch = statebatch.reshape(statebatch.shape[0],int(statebatch.shape[1] /4),4)
actionseq = np.array(['H' if i[0]>0.5 else 'B' if i[1]>0.5 else 'S' for i in actionbatch ])

In [55]:
idxWhere =np.where(rewardbatch == np.max(rewardbatch))[0][0]
idxWhere

1624

In [57]:
state = statebatch[idxWhere]
header = state[:1]
# pos = state[1:2]
# time = state[2:3]
KL1m = state[1:(1+30)]
KL5m = state[31:(31+96)]
KL1d = state[(31+96):]
state

array([[  6.       ,   5.       ,   3.       , 900.       ],
       [  1.011612 ,   2.8174725,   1.0122951,   1.010929 ],
       [  1.011612 ,   1.2793343,   1.011612 ,   1.010929 ],
       ...,
       [  1.2356558,   0.9726144,   1.2452186,   1.2295082],
       [  1.2424864,   1.2889742,   1.2452186,   1.2288251],
       [  1.2349727,   1.1519059,   1.2465847,   1.2240437]],
      dtype=float32)

In [58]:
DataFrame(header,columns=['mon','day','wday','min'])

,mon,day,wday,min
0,6.0,5.0,3.0,900.0


In [59]:
DataFrame(KL1m,columns=['P','V','H','L'])

,P,V,H,L
0,1.011612,2.817472,1.012295,1.010929
1,1.011612,1.279334,1.011612,1.010929
2,1.011612,1.674522,1.012295,1.011612
3,1.011612,2.398932,1.011612,1.011612
4,1.010929,1.423422,1.011612,1.010929
5,1.011612,0.733077,1.011612,1.010929
6,1.011612,1.338438,1.012295,1.011612
7,1.011612,0.628352,1.012295,1.011612
8,1.012295,4.015073,1.012295,1.011612
9,1.012295,6.094653,1.012295,1.010929


In [60]:
DataFrame(KL5m,columns=['P','V','H','L'])

,P,V,H,L
0,1.011612,1.342988,1.012295,1.010929
1,1.010929,1.850966,1.011612,1.010246
2,1.010246,0.904490,1.010929,1.008880
3,1.009563,1.066176,1.010246,1.009563
4,1.010929,5.082886,1.012295,1.008197
5,1.008197,0.995517,1.008880,1.007514
6,1.009563,2.296976,1.009563,1.006148
7,1.006148,0.401519,1.006148,1.005464
8,1.006831,1.466540,1.006831,1.005464
9,1.005464,1.943485,1.005464,1.004098


In [61]:
DataFrame(KL1d,columns=['P','V','H','L'])

,P,V,H,L
0,1.000000,1.000000,1.006831,0.995902
1,1.006148,1.061635,1.016393,1.006148
2,1.008880,1.165055,1.014344,1.006831
3,1.012295,1.333900,1.019809,1.011612
4,1.016393,1.503773,1.017076,1.004098
5,1.009563,1.129486,1.015027,1.008197
6,1.012295,1.279374,1.019126,1.008880
7,1.010929,1.488795,1.012295,1.002049
8,1.001366,1.655389,1.017760,1.000683
9,1.003415,1.510816,1.004781,0.985656


In [32]:
# actionseq
# actionseq[idxWhere-10:idxWhere+10]
idxWhere = np.where(actionseq=='B')[0][5]
actionseq[idxWhere-20:idxWhere+300]

array(['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'B', 'B', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'B', 'B', 'H', 'B', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'B',
       'B', 'H', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'H', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B',
       'B', 'B', 'B', 'B', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'B', 'B', 'B', 'B', 'B', 'B', 'H',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'H', 'B', 'B', 'B',
       'H', 'B', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B

In [33]:
rewardbatch[idxWhere-10:idxWhere+100]

array([ 0.5      ,  3.5193372, 10.556228 ,  7.029742 ,  1.       ,
        1.       ,  3.5131536,  1.       ,  0.5      ,  0.5      ,
        0.5      ,  0.5      ,  0.5      ,  0.5      ,  0.5      ,
       17.606749 ,  7.029742 ,  1.       ,  1.       ,  1.       ,
        7.0322137, 14.055507 ,  1.       ,  1.       ,  3.5094538,
       14.035778 ,  1.       ,  1.       ,  1.       ,  1.       ,
        0.5      ,  0.5      , 14.080247 ,  0.5      , 10.559943 ,
        1.       , 35.16493  ,  1.       ,  1.       ,  1.       ,
        0.5      ,  0.5      ,  0.5      , 14.090167 ,  0.5      ,
        0.5      ,  0.5      ,  0.5      ,  0.5      ,  0.5      ,
        0.5      ,  0.5      ,  0.5      ,  0.5      ,  0.5      ,
        0.5      ,  0.5      ,  0.5      ,  0.5      ,  0.5      ,
        1.       ,  0.5      ,  0.5      ,  0.5      ,  0.5      ,
        0.5      ,  0.5      ,  0.5      , 24.667208 ,  7.029742 ,
        1.       ,  1.       ,  1.       ,  1.       ,  3.5131